In [2]:
import plotly
import plotly.graph_objects as go
from random import randint, choice

import dice_game

In [22]:
# def face_rotation(face_num, direction, face_dict):
    
#     if direction == 'right':
#         change_dict = {
#             'up':'right',
#             'right' :'down',
#             'down': 'left',
#             'left':'up'
#             }
        
#     elif direction == 'left':
#         change_dict = {
#             'up':'left',
#             'left' :'down',
#             'down': 'right',
#             'right':'up'
#             }
    
#     replacement_dict = {}
    
#     for key, val in face_dict.items():
#             new_key = change_dict[key]
#             val_num = val.split(' ')[0]
#             val_direction = val.split(' ')[1]
#             new_direction = change_dict[val_direction]
#             replacement_dict[new_key] = val_num + ' ' + new_direction
    
#     return replacement_dict

# rotation_dict = {}
# for key in ['1 up', '2 up', '3 up', '4 up', '5 up', '6 up']:
#     num = key.split(' ')[0]
#     rotation_dict[key] = move_dict[key]
#     rotation_dict[f'{num} right'] = face_rotation(num, 'right', rotation_dict[key])
#     rotation_dict[f'{num} down'] = face_rotation(num, 'right', rotation_dict[f'{num} right'])
#     rotation_dict[f'{num} left'] = face_rotation(num, 'right', rotation_dict[f'{num} down'])

In [3]:
rotation_dict = {'1 up': {'up': '2 up',
  'down': '5 down',
  'left': '4 left',
  'right': '3 right'},
 '1 right': {'right': '2 right',
  'left': '5 left',
  'up': '4 up',
  'down': '3 down'},
 '1 down': {'down': '2 down',
  'up': '5 up',
  'right': '4 right',
  'left': '3 left'},
 '1 left': {'left': '2 left',
  'right': '5 right',
  'down': '4 down',
  'up': '3 up'},
 '2 up': {'up': '6 up', 'down': '1 up', 'left': '3 up', 'right': '4 up'},
 '2 right': {'right': '6 right',
  'left': '1 right',
  'up': '3 right',
  'down': '4 right'},
 '2 down': {'down': '6 down',
  'up': '1 down',
  'right': '3 down',
  'left': '4 down'},
 '2 left': {'left': '6 left',
  'right': '1 left',
  'down': '3 left',
  'up': '4 left'},
 '3 up': {'up': '6 right', 'down': '1 left', 'left': '2 up', 'right': '5 up'},
 '3 right': {'right': '6 down',
  'left': '1 up',
  'up': '2 right',
  'down': '5 right'},
 '3 down': {'down': '6 left',
  'up': '1 right',
  'right': '2 down',
  'left': '5 down'},
 '3 left': {'left': '6 up',
  'right': '1 down',
  'down': '2 left',
  'up': '5 left'},
 '4 up': {'up': '6 left', 'down': '1 right', 'left': '5 up', 'right': '2 up'},
 '4 right': {'right': '6 up',
  'left': '1 down',
  'up': '5 right',
  'down': '2 right'},
 '4 down': {'down': '6 right',
  'up': '1 left',
  'right': '5 down',
  'left': '2 down'},
 '4 left': {'left': '6 down',
  'right': '1 up',
  'down': '5 left',
  'up': '2 left'},
 '5 up': {'up': '6 down', 'down': '1 down', 'left': '3 up', 'right': '4 up'},
 '5 right': {'right': '6 left',
  'left': '1 left',
  'up': '3 right',
  'down': '4 right'},
 '5 down': {'down': '6 up', 'up': '1 up', 'right': '3 down', 'left': '4 down'},
 '5 left': {'left': '6 right',
  'right': '1 right',
  'down': '3 left',
  'up': '4 left'},
 '6 up': {'up': '5 down',
  'down': '2 up',
  'left': '4 right',
  'right': '3 left'},
 '6 right': {'right': '5 left',
  'left': '2 right',
  'up': '4 down',
  'down': '3 up'},
 '6 down': {'down': '5 up',
  'up': '2 down',
  'right': '4 left',
  'left': '3 right'},
 '6 left': {'left': '5 right',
  'right': '2 left',
  'down': '4 up',
  'up': '3 down'}}

In [4]:
def get_starting_position(grid_size):
    starting_pos = [randint(1,grid_size),randint(1,grid_size)]
    starting_face = choice([x for x in rotation_dict])
        
    return starting_pos, starting_face

def make_path(grid_size, path_len):    
    
    starting_pos, starting_face = get_starting_position(grid_size)
    current_pos = starting_pos[:]
    current_face = starting_face
    path_trace = [starting_pos]
    current_path_len=1
    
    move_attempt = 1
    
    while current_path_len < path_len:
        new_pos, new_face = single_move(current_pos, current_face, grid_size)
        
        if move_attempt>20:
            break
            
        if new_pos in path_trace:
            move_attempt+=1
            continue
        else:
            tries=1
            path_trace.append(new_pos)
            current_pos = new_pos
            current_face = new_face
            current_path_len+=1
            
    return starting_pos, starting_face, current_pos, current_face, path_trace

def single_move(current_pos, current_face, grid_size):
    new_pos = current_pos[:]
    oob_options = set()
    
    if current_pos[0] == 1:
        oob_options.add('left')
    elif current_pos[0] == grid_size:
        oob_options.add('right')
    if current_pos[1] == 1:
        oob_options.add('down')
    elif current_pos[1] == grid_size:
        oob_options.add('up')
    
    legit_moves = {'up','right','left','down'} - oob_options
    chosen_move = choice(list(legit_moves))
    if chosen_move == 'up':
        new_pos[1] +=1
    elif chosen_move == 'down':
        new_pos[1] -=1
    elif chosen_move == 'right':
        new_pos[0] +=1
    elif chosen_move == 'left':
        new_pos[0] -=1

    new_face = rotation_dict[current_face][chosen_move]

    return new_pos, new_face

In [5]:
def make_fig(grid_size):
    fig = go.Figure()
    fig.layout.xaxis.visible=True
    fig.layout.yaxis.visible=True
    fig.layout.xaxis.range=(0,grid_size)
    fig.layout.yaxis.range=(0,grid_size)
    fig.layout.height=600
    fig.layout.width=600
    fig.update_yaxes(nticks=grid_size*2)
    fig.update_xaxes(nticks=grid_size*2)
    return fig

def add_line(fig, start_pos, end_pos, color, offset=0):
    fig.add_shape(
        dict(
            type="line",
            x0=start_pos[0]-0.5-offset,
            y0=start_pos[1]-0.5-offset,
            x1=end_pos[0]-0.5-offset,
            y1=end_pos[1]-0.5-offset,
            line=dict(
                color=color,
                width=3
            )))

def add_path(fig, path_traces, colors):
    offset = 0
    
    for i, path_trace in enumerate(path_traces):
        if not path_trace:
            continue
        color = colors[i]
        for j, step in enumerate(path_trace):
            if j<len(path_trace)-1:
                add_line(fig, step, path_trace[j+1], color, offset)
                
        offset+=0.1       

def add_labels(fig, annotation_set, colors):

    def get_rotation(face):
        if 'up' in face:
            rotation = 0
        elif 'right' in face:
            rotation = 90
        if 'down' in face:
            rotation = 180
        if 'left' in face:
            rotation = 270
        return rotation
    
    offset=0
    annotations = []
    
    for i, label_set in enumerate(annotation_set):
        starting_pos = label_set[0]
        starting_face = label_set[1]
        end_pos = label_set[2]
        end_face = label_set[3]
        
        annotations.append(
            go.layout.Annotation(
                x=starting_pos[0]-0.5 - offset,
                y=starting_pos[1]-0.5 - offset,
                text=f"<b>{starting_face.split(' ')[0]}</b>",
                align='center',
                showarrow=False,
                font={'size':38, 'color':colors[i]},
                textangle=get_rotation(starting_face)
            )
        )
        
        annotations.append(
            go.layout.Annotation(
                    x=end_pos[0]-0.5 - offset,
                    y=end_pos[1]-0.5 - offset,
                    text=f"<b>{end_face.split(' ')[0]}</b>",
                    align='center',
                    showarrow=False,
                    font={'size':38, 'color':colors[i]},
                    textangle=get_rotation(end_face)
            )
        )
    offset += 0.1

    
    fig.update_layout(
        annotations=annotations
    )
    
    return fig

def shift_solution_make_incorrect(grid_size, current_pos):
    oob_options = set()
    new_pos = current_pos
    
    if current_pos[0] == 1:
        oob_options.add('left')
    elif current_pos[0] == grid_size:
        oob_options.add('right')
    if current_pos[1] == 1:
        oob_options.add('down')
    elif current_pos[1] == grid_size:
        oob_options.add('up')
    
    legit_moves = {'up','right','left','down'} - oob_options
    chosen_move = choice(list(legit_moves))
    if chosen_move == 'up':
        new_pos[1] +=1
    elif chosen_move == 'down':
        new_pos[1] -=1
    elif chosen_move == 'right':
        new_pos[0] +=1
    elif chosen_move == 'left':
        new_pos[0] -=1
        
    return new_pos

def get_fake_solution(grid_size, path_len):
    starting_pos, starting_face, current_pos, current_face, path_trace = make_path(grid_size, path_len)
    current_pos = shift_solution_make_incorrect(grid_size, current_pos)
    return starting_pos, starting_face, current_pos, current_face


In [6]:
def add_labels_from_map(fig, map_, colors):
    """
    same as the other one, but now takes input from Greg's cool code.
    """
    offset=0
    annotations = []
    
    for i, face_pair in enumerate(map_['pairs']):
        starting_pos = face_pair[0]['pos']
        starting_face = face_pair[0]['face']
        starting_rotation = face_pair[0]['angle']
        end_pos = face_pair [1]['pos']
        end_face = face_pair[1]['face']
        end_rotation = face_pair[1]['angle']
        
        annotations.extend([
            # Start and end faces
            go.layout.Annotation(
                x=starting_pos[0]-0.5 - offset,
                y=starting_pos[1]-0.5 - offset,
                text=f"<b>{str(starting_face)}</b>",
                align='center',
                showarrow=False,
                font={'size':38, 'color':colors[i], 'family':'acumen pro'},
                textangle=starting_rotation
            ),
            go.layout.Annotation(
                x=end_pos[0]-0.5 - offset,
                y=end_pos[1]-0.5 - offset,
                text=f"<b>{str(end_face)}</b>",
                align='center',
                showarrow=False,
                font={'size':38, 'color':colors[i], 'family':'acumen pro'},
                textangle=end_rotation
            ),
            # Set Labels
            go.layout.Annotation(
                x=starting_pos[0]-0.9 - offset,
                y=starting_pos[1] - 0.1 - offset,
                text=f"{str(i+1)}",
                align='center',
                showarrow=False,
                font={'size':16, 'color':'black', 'family':'acumen pro'},
                textangle=0
            ),
            go.layout.Annotation(
                x=end_pos[0]-0.9 - offset,
                y=end_pos[1]-0.1 - offset,
                text=f"{str(i+1)}",
                align='center',
                showarrow=False,
                font={'size':16, 'color':'black', 'family':'acumen pro'},
                textangle=0

            )         
        ])
        
#         for block in map_['blocks']:
#             annotations.append(
#                 go.layout.Annotation(
#                     x=block[0] - 0.5,
#                     y=block[1] - 0.5,
#                     text="X",
#                     align='center',
#                     showarrow=False,
#                     font={'size':25, 'color':'red', 'family':'helvetica neue'},
#                     textangle=0

#                 )
#             )
            
    offset += 0.1

    
    fig.update_layout(
        annotations=annotations
    )
    
    return fig

In [7]:
def add_blocks(fig, map_):
    
    for block in map_['blocks']:
             fig.add_shape(
                 type="rect",
                 x0 = block[0] - 1,
                 y0 = block[1] - 1,
                 x1 = block[0],
                 y1 = block[1],
                 fillcolor="#999999",
                 line={'color':'#666666','width':1},
                 )
    
#                 go.layout.Annotation(
#                     x=block[0] - 0.5,
#                     y=block[1] - 0.5,
#                     text="X",
#                     align='center',
#                     showarrow=False,
#                     font={'size':25, 'color':'red', 'family':'helvetica neue'},
#                     textangle=0

#                 )
#             )

In [8]:
def make_grid(grid_size, real_solutions, fake_solutions, path_length_min, path_length_max):
    
    reals = ['#C3A067','#bb9394','#004b65','#414042','red','green','blue']
    fakes = ['black','blue','ForestGreen']
    colors = reals[:real_solutions] + fakes[:fake_solutions] 
    
    fig = make_fig(grid_size)
    
    label_sets = []
    path_traces = []
    used_squares = []
    traces_added = 0
    fake_traces_added = 0
    while traces_added < real_solutions:
        starting_pos, starting_face, end_pos, end_face, path_trace = make_path(grid_size, path_len=randint(path_length_min,path_length_max))      
        
        if starting_pos in used_squares or end_pos in used_squares:
            continue
        else:
            label_sets.append([starting_pos, starting_face, end_pos, end_face])
            used_squares.extend([starting_pos, end_pos])
            path_traces.append(path_trace)
            traces_added+=1
            
    while fake_traces_added < fake_solutions:
        starting_pos, starting_face, end_pos, end_face = get_fake_solution(grid_size, path_len=randint(path_length_min,path_length_max))
        
        if starting_pos in used_squares or end_pos in used_squares:
            continue
        
        else:
            label_sets.append([starting_pos, starting_face, end_pos, end_face])
            used_squares.extend([starting_pos, end_pos])
            fake_traces_added+=1

    add_path(fig, path_traces, colors)
    add_labels(fig, label_sets, colors)
    
    return fig

In [9]:
def map_map(map_, grid_size):
    
    colors =  ['#C3A067','#bb9394','#004b65','#414042'] * 3
    
    fig = make_fig(grid_size)
    
    add_path(fig, map_['solutions'], colors)
    add_labels_from_map(fig, map_, colors)
    add_blocks(fig, map_)
    
    return fig

In [10]:
def save_grid(n, grid_size, pair_count, blocks=False, min_length=2, fpath='images/'):
    for i in range(n):

        dice_game.GRID_SIZE= grid_size
        
        map_ = dice_game.generate_map(
            pair_count=pair_count,
            add_block=blocks,
            min_length=min_length,
            strict_impossible=False
            )
        fig = map_map(map_, grid_size=dice_game.GRID_SIZE)
        fig.write_image(f"{fpath}{dice_game.GRID_SIZE}x{dice_game.GRID_SIZE}-{pair_count}-{i}", format='pdf')

In [11]:
save_grid(20, 4, 2, False, 3)

ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Program Files (x86)\Common Files\Intel\Shared Libraries\redist\ia32\compiler
    C:\Program Files (x86)\Common Files\Intel\Shared Libraries\redist\intel64\compiler
    C:\Program Files\Oculus\Support\oculus-runtime
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common
    C:\Program Files (x86)\Calibre2\
    C:\Users\robmd\AppData\Local\Programs\Python\Python38-32\Scripts\
    C:\Users\robmd\AppData\Local\Programs\Python\Python38-32\
    C:\Users\robmd\AppData\Local\Microsoft\WindowsApps
    C:\Users\robmd\AppData\Local\GitHubDesktop\bin
    C:\Users\robmd\AppData\Local\Programs\Microsoft VS Code\bin

If you haven't installed orca yet, you can do so

In [255]:
dice_game.GRID_SIZE=4

map_ = dice_game.generate_map(
    pair_count=2,
    add_block=True,
    min_length=2,
    strict_impossible=False
    )

fig = map_map(map_, grid_size=dice_game.GRID_SIZE)

fig

In [40]:
fig = make_grid(
    grid_size=4,
    real_solutions=2,
    fake_solutions=2,
    path_length_min=5,
    path_length_max=8
)

In [17]:
for i in range(30):
    paths=2
    reals = randint(0,paths)
    
    fig = make_grid(
        grid_size=4,
        real_solutions=reals,
        fake_solutions=paths-reals,
        path_length_min=4,
        path_length_max=8
        )
    fig.write_image(f"images/4x4_{paths}paths_{i}", format='pdf')

In [18]:
for i in range(30):
    paths=2
    grid=3
    reals = randint(0,paths)
    
    fig = make_grid(
        grid_size=grid,
        real_solutions=reals,
        fake_solutions=paths-reals,
        path_length_min=3,
        path_length_max=5
        )
    fig.write_image(f"images/{grid}x{grid}_{paths}paths_{i}", format='pdf')